In [1]:
import os, sys

notebook_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_dir,'..')))

In [2]:
from scripts.data_loader import load_data
from scripts.data_loader import load_data_with_parse
from textblob import TextBlob

In [10]:

aapl_data = load_data_with_parse("../data/yfinance_data/AAPL_historical_data.csv", parse_dates=['Date'])
news_data = load_data_with_parse("../data/raw_analyst_ratings.csv", parse_dates=['Date'])


In [12]:
news_data.head()

,Unnamed: 0,headline,url,publisher,date,stock
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A


In [8]:
stock_data = aapl_data.sort_values(by='Date').reset_index(drop=True)
news_data = news_data.sort_values(by='Date').reset_index(drop=True)

KeyError: 'Date'